In [ ]:
! git clone https://github.com/sjmoran/CURL

% cd ./CURL/

Cloning into 'CURL'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 517 (delta 44), reused 6 (delta 2), pack-reused 436
Receiving objects: 100% (517/517), 92.68 MiB | 33.49 MiB/s, done.
Resolving deltas: 100% (281/281), done.
/content/CURL


# Codes

In [ ]:
%cd ./CURL/

import numpy as np
from PIL import Image
import sys
from util import ImageProcessing

import torch
import torchvision.transforms.functional as TF
import util

import requests
from io import BytesIO
import model

np.set_printoptions(threshold=sys.maxsize)

import matplotlib.pyplot as plt
%matplotlib inline


checkpoint_filepath = "./pretrained_models/adobe_dpe/curl_validpsnr_23.073045286204017_validloss_0.0701291635632515_testpsnr_23.584083321292365_testloss_0.061363041400909424_epoch_510_model.pt"
inference_img_dirpath = "./adobe5k_dpe"
log_dirpath = './'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

net = model.CURLNet()
checkpoint = torch.load(checkpoint_filepath, map_location=DEVICE)
net.load_state_dict(checkpoint['model_state_dict'])

net.eval()
if DEVICE == 'cuda':
  net.cuda()


def load_image(path):
    '''
    '''
    if ("https" in path) or ("http" in path):
      image = Image.open(BytesIO(requests.get(path).content))

    else:
      image = Image.open(path)


    if image.mode != 'RGB':
      image = image.convert('RGB')
               
    return TF.to_tensor(image).to(DEVICE)


def evaluate(img_path, convert_uint = False):
    """
    """
    img = load_image(img_path)

    with torch.no_grad():

        img = img.unsqueeze(0)
        img = torch.clamp(img, 0, 1)

        net_output_img_example , _ = net(img)

        net_output_img_example_numpy = net_output_img_example.squeeze(0).data.cpu().numpy()
        net_output_img_example_numpy = ImageProcessing.swapimdims_3HW_HW3(net_output_img_example_numpy)
        return (net_output_img_example_numpy* 255).astype(np.uint8) if convert_uint else net_output_img_example_numpy

        

[Errno 2] No such file or directory: './CURL/'
/content/CURL


In [ ]:
path = "./adobe5k_dpe/curl_example_test_input/a2803-060810_075208_GM6A0020_input.png"
result = evaluate(path, convert_uint = False)

f, ax = plt.subplots(1,2, figsize = (20,10))
ax[0].imshow(np.array(Image.open(path)))
ax[1].imshow(result)

In [ ]:
url = "https://2.img-dpreview.com/files/p/TS560x560~forums/62902641/8ec07adb32b04f5dababb8f24bf10fb3"
result = evaluate(url, convert_uint = False)

f, ax = plt.subplots(1,2, figsize = (20,10))
ax[0].imshow(np.array(Image.open(BytesIO(requests.get(url).content))))
ax[1].imshow(result)